# Using SQL's window functions to analyze tech stock prices through Google BigQuery

### Author: Antonio Rueda-Toicen
### antonio.rueda.toicen@gmail.com


### Introduction to Window Functions

Window functions, often referred to as analytic or OLAP (Online Analytical Processing) functions, allow us to perform calculations across a set of table rows that are related to the current row. They allow us to do aggregations, like we do with GROUP BY.

However, unlike GROUP BY. which return a single value for each group, window functions return multiple values. This can be especially useful when you need to maintain the detail level of your dataset while performing such computations.

We can also do operations that go beyond aggregation with window functions. With them we can perform rank and lead/lag operations on within the subset that we have defined.

#### Key Components of Window Functions:

1. **Function**: This is the actual computation, like `AVG`, `SUM`, `MAX`, `LEAD` etc.
2. **Partitioning**: This divides the result set into partitions and the window function is applied to each partition. For example, if you partition by `Symbol` (the name of a traded company), then each stock symbol will have its own set of calculations.
3. **Ordering**: Rows in a partition can be ordered using the `ORDER BY` clause. This is crucial for functions like running totals or moving averages.
4. **Frame Specification**: This defines which rows are included in the frame for each row. For instance, "rows between 1 preceding and 1 following" would include the current row, the previous row, and the next row. This would allow us to compute trailing or centered moving averages.

### Use Cases with Our Dataset:

1. **Moving Averages**: As demonstrated earlier, window functions can compute moving averages for stock prices. For instance, a five-day trailing moving average for the `Close` column. This is an important signal for us to know whether a particular stock is trending up or down.
   
2. **Cumulative Volume**: To understand the cumulative trading volume over time for a particular stock symbol:
   ```sql
   SUM(Volume) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Cumulative_Volume
   ```

3. **Maximum and Minimum Close Price**: To get the highest and lowest `Close` price for each stock symbol till a particular date:
   ```sql
   MAX(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Max_Close_Till_Date,
   MIN(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Min_Close_Till_Date
   ```

4. **Lag and Lead**: To compare the `Close` price of a stock with its previous day's price or its price from a day ahead:
   ```sql
   LAG(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Previous_Day_Close,
   LEAD(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Next_Day_Close
   ```

5. **Row Numbering**: To assign a unique sequential integer to rows within a partition of a result set, which can be useful for tasks like identifying the nth highest value:
   ```sql
   ROW_NUMBER() OVER (PARTITION BY Symbol ORDER BY Close DESC) AS Rank_By_Close_Price
   ```

In summary, window functions offer a powerful way to perform calculations that need to consider a "window" of rows relative to the current row, without collapsing all the rows into a single output row. They're versatile, and understanding them can greatly expand the range of data processing tasks you can handle with SQL. Given your interest in computer vision and machine learning, window functions can be particularly useful when preprocessing time-series data or any sequential data before feeding it into a model.


## Analysing stock market data
In this notebook, we analyse trends on the stock market through the use of SQL window functions. We will explore how to compute moving averages, which are useful to see the overall trend of a stock's price through time.

We have obtained this data from the Yahoo Finance package through [this notebook](https://colab.research.google.com/drive/1YM2R96ONl5SE85nquiNXx_HX-WAWwMa-#scrollTo=doYg2iyXsrz9)
and [uploaded it to Bigquery](https://cloud.google.com/bigquery/docs/datasets) as a CSV file.

## Schema

Our data has the following schema, which has been auto-detected during the upload:

```
TradeDate	 DATE	   NULLABLE
Symbol	   STRING	  NULLABLE
Close	     FLOAT	  NULLABLE
Volume	   INTEGER	 NULLABLE
```

`TradeDate` corresponds to the trading day, `Symbol` corresponds to the name of the tech company, `Close` is its closing price, and `Volume` is the number of transactions.

The transactions correspond to October 2022- October 2023.

### Authenticating with Google Cloud Platform

We need to use here same account that we are using on the Google Cloud Console.

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Authenticating with Google Cloud Platform to access BigQuery

We need to have a project and dataset created on Bigquery in order to access the upload the data. Refer to [this page of documentation](https://cloud.google.com/bigquery/docs/datasets) for guidance.

### Specifying the `project_id`

To access our tables, we will need to provide the `project_id` and the dataset name. Refer to [this page of documentation on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects
) to learn more about the creation of projects.

In [ ]:
import pandas as pd

# this import makes our tables look nicer on our notebook
from google.colab import data_table
data_table.enable_dataframe_formatter()


# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = f'focal-cache-403012'
project_id

'focal-cache-403012'

### Defining the fully qualified table name

`stock_prices` is a dataset that we can create when first upload the CSV file. It remains available for us to add more tables to it later.

In [ ]:
# Specifying the full name of the table requires specifying
# project_id.dataset_id.table_name.
# In our case, the dataset is called stocks
stocks_table = f'{project_id}.stock_prices.tech_stocks'
stocks_table

'focal-cache-403012.stock_prices.tech_stocks'

In [ ]:
import pandas as pd

project_id = 'focal-cache-403012'

df = pd.io.gbq.read_gbq(f'''
  SELECT * FROM `focal-cache-403012.stock_prices.tech_stocks`
    LIMIT 1000
''', project_id=project_id)

df

,TradeDate,Symbol,Close,Volume
0,2022-10-27,AAPL,144.800003,109180200
1,2022-10-28,AAPL,155.740005,164762400
2,2022-10-31,AAPL,153.339996,97943200
3,2022-11-01,AAPL,150.649994,80379300
4,2022-11-02,AAPL,145.029999,93604600
...,...,...,...,...
995,2023-10-16,META,321.149994,16536100
996,2023-10-17,META,324.000000,16387800
997,2023-10-18,META,316.970001,16851000
998,2023-10-19,META,312.809998,18709200


## Understanding window functions by example

## Creating a five-day moving average

To compute a five-day trailing moving average of the `Close` column in Google BigQuery, we'd typically use a window function with the `AVG()` aggregate function and the `ROWS BETWEEN` clause to specify the window frame.

Here's a step-by-step breakdown:

1. **Window Specification**: We define a window over which the moving average is computed. In this case, it's a trailing window that includes the current row and the four previous rows.

2. **Aggregate Function**: We use the `AVG()` function to compute the average of the values in the window.

3. **Partitioning and Ordering**: We partition the data by `Symbol` to compute the moving average separately for each stock symbol. We also order the rows by `TradeDate` to ensure the average is computed in the right sequence of dates.

Given these considerations, here's the SQL query to compute the five-day trailing moving average for the `Close` column using Google BigQuery:

```sql
SELECT
  TradeDate,
  Symbol,
  Close,
  AVG(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS Five_Day_Trailing_Avg
FROM
  stocks_table
ORDER BY
  Symbol, TradeDate;
```

Replace `stocks_table` with the name of your table using a format string. This query will give you the original `TradeDate`, `Symbol`, and `Close` columns, along with a new column `Five_Day_Trailing_Avg` that contains the five-day trailing moving average for the `Close` column.

In [ ]:
df = pd.io.gbq.read_gbq(f'''
  SELECT
  TradeDate,
  Symbol,
  Close,
  AVG(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS Five_Day_Trailing_Avg
FROM
  {stocks_table}
ORDER BY
  Symbol, TradeDate;

''', project_id=project_id)

df

,TradeDate,Symbol,Close,Five_Day_Trailing_Avg
0,2022-10-27,AAPL,144.800003,144.800003
1,2022-10-28,AAPL,155.740005,150.270004
2,2022-10-31,AAPL,153.339996,151.293335
3,2022-11-01,AAPL,150.649994,151.132500
4,2022-11-02,AAPL,145.029999,149.912000
...,...,...,...,...
1501,2023-10-20,NVDA,413.869995,431.434003
1502,2023-10-23,NVDA,429.750000,425.194000
1503,2023-10-24,NVDA,436.630005,424.644000
1504,2023-10-25,NVDA,417.790009,423.810004


In [ ]:
import plotly.express as px

# Create a line plot
fig = px.line(df,
              x='TradeDate',
              y=['Five_Day_Trailing_Avg'],
              color='Symbol',
              facet_row='Symbol',
              title='5-Day Trailing Average',
              labels={'Close': 'Close Price', 'Five_Day_Trailing_Avg': '5-Day Avg'},
              template='plotly_dark')

fig.for_each_trace(lambda trace: trace.update(name=trace.name.split('=')[-1].strip()))


# Show the plot
fig.show()


## Calculating cumulative volume of transactions


To compute the cumulative volume for each `Symbol`:

```sql
SELECT
  TradeDate,
  Symbol,
  Close,
  Volume,
  SUM(Volume) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Cumulative_Volume
FROM
  stocks_table
ORDER BY
  Symbol, TradeDate;
```
Here's a step-by-step breakdown:

1. **Aggregate Function**: We use the `SUM()` function to compute the cumulative sum of the `Volume` column.

2. **Partitioning**: We partition the data by `Symbol` to compute the cumulative volume separately for each stock symbol.

3. **Ordering**: Rows are ordered by `TradeDate` to ensure the cumulative sum is computed in chronological order.


Let's try this through Pandas's interface with BigQuery:

In [28]:
df = pd.io.gbq.read_gbq(f'''
 SELECT
  TradeDate,
  Symbol,
  Close,
  Volume,
  SUM(Volume) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Cumulative_Volume
FROM
  {stocks_table}
ORDER BY
  Symbol, TradeDate;

''', project_id=project_id)

df

,TradeDate,Symbol,Close,Volume,Cumulative_Volume
0,2022-10-27,AAPL,144.800003,109180200,109180200
1,2022-10-28,AAPL,155.740005,164762400,273942600
2,2022-10-31,AAPL,153.339996,97943200,371885800
3,2022-11-01,AAPL,150.649994,80379300,452265100
4,2022-11-02,AAPL,145.029999,93604600,545869700
...,...,...,...,...,...
1501,2023-10-20,NVDA,413.869995,47638100,11981224700
1502,2023-10-23,NVDA,429.750000,47853000,12029077700
1503,2023-10-24,NVDA,436.630005,40146300,12069224000
1504,2023-10-25,NVDA,417.790009,39837900,12109061900


### Maximum and minimum closing prices

To compute the maximum and minimum closing prices up to a given date using window functions, we'd use the `MAX()` and `MIN()` functions, respectively.

Here's how we can do it:

### SQL Query:
```sql
SELECT
  TradeDate,
  Symbol,
  Close,
  MAX(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Max_Close_Up_To_Date,
  MIN(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Min_Close_Up_To_Date
FROM
  your_table_name
ORDER BY
  Symbol, TradeDate;
```
Replace `your_table_name` with the actual name of your table.

### Explanation:

1. **Aggregate Functions**:
   - `MAX(Close)`: This computes the maximum closing price.
   - `MIN(Close)`: This computes the minimum closing price.

2. **Partitioning**: We partition the data by `Symbol` to compute the max and min separately for each stock symbol. This ensures that the values from one stock don't influence the values of another.

3. **Ordering**: The rows are ordered by `TradeDate` in ascending order. This means for each row, the window includes all previous dates and the current date for that stock symbol. Hence, the max and min values represent the highest and lowest closing prices up to and including that date.

4. **Frame Specification**: By default, if not specified, the frame starts at the first row of the partition and goes up to the current row, because of the ascending order specified in the `ORDER BY` clause.

In summary, this query will give us, for each date and stock symbol, the closing price of the stock (`Close`), and the highest (`Max_Close_Up_To_Date`) and lowest (`Min_Close_Up_To_Date`) closing prices seen for that stock up to and including that date.

In [29]:
df = pd.io.gbq.read_gbq(f'''
SELECT
  TradeDate,
  Symbol,
  Close,
  MAX(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Max_Close_Up_To_Date,
  MIN(Close) OVER (PARTITION BY Symbol ORDER BY TradeDate ASC) AS Min_Close_Up_To_Date
FROM
  {stocks_table}
ORDER BY
  Symbol, TradeDate;


''', project_id=project_id)

df

,TradeDate,Symbol,Close,Max_Close_Up_To_Date,Min_Close_Up_To_Date
0,2022-10-27,AAPL,144.800003,144.800003,144.800003
1,2022-10-28,AAPL,155.740005,155.740005,144.800003
2,2022-10-31,AAPL,153.339996,155.740005,144.800003
3,2022-11-01,AAPL,150.649994,155.740005,144.800003
4,2022-11-02,AAPL,145.029999,155.740005,144.800003
...,...,...,...,...,...
1501,2023-10-20,NVDA,413.869995,493.549988,131.759995
1502,2023-10-23,NVDA,429.750000,493.549988,131.759995
1503,2023-10-24,NVDA,436.630005,493.549988,131.759995
1504,2023-10-25,NVDA,417.790009,493.549988,131.759995


### Lag and lead prices

The `LAG()` and `LEAD()` functions are window functions that allow us to access data from a previous row (`LAG()`) or a subsequent row (`LEAD()`) in the result set, without a [self-join](https://www.w3schools.com/sql/sql_join_self.asp).

Here's how we can compute the lag (previous day's) and lead (next day's) prices for the `Close` column:

### SQL Query:
```sql
SELECT
  TradeDate,
  Symbol,
  Close,
  LAG(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Previous_Day_Close,
  LEAD(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Next_Day_Close
FROM
  tech_stocks
ORDER BY
  Symbol, TradeDate;
```

This query provides, for each date and stock symbol, the closing price of the stock (`Close`), the closing price of the stock from the previous day (`Previous_Day_Close`), and the closing price of the stock for the next day (`Next_Day_Close`). If there's no previous or next day data (e.g., for the first and last dates in the dataset), the respective lag or lead value will be `NULL`.


### Step-by-Step Explanation:

1. **Window Functions**:
   - `LAG(Close, 1)`: The `LAG()` function fetches the value of the `Close` column from the previous row. The second argument `1` indicates how many rows back we want to go. If you omit this argument, it defaults to 1.
   - `LEAD(Close, 1)`: The `LEAD()` function fetches the value of the `Close` column from the subsequent row. The second argument `1` indicates how many rows ahead we want to go. Again, if you omit this argument, it defaults to 1.

2. **Partitioning**: We partition the data by `Symbol` to compute the lag and lead separately for each stock symbol. This ensures that the values from one stock don't influence the values of another and that the lag and lead values are correctly computed within each stock symbol's data.

3. **Ordering**: The rows are ordered by `TradeDate`. This ensures that the data is processed chronologically, so the previous day's price is fetched correctly with `LAG()` and the next day's price with `LEAD()`.

4. **Frame Specification**: By default, for `LAG()` and `LEAD()`, the frame consists of the current row's position minus or plus the specified offset (1 in this case). No need for explicit frame specification.



In [32]:
df = pd.io.gbq.read_gbq(f'''
SELECT
  TradeDate,
  Symbol,
  Close,
  LAG(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Previous_Day_Close,
  LEAD(Close, 1) OVER (PARTITION BY Symbol ORDER BY TradeDate) AS Next_Day_Close
FROM
  {stocks_table}
ORDER BY
  Symbol, TradeDate;
''', project_id=project_id)

df

,TradeDate,Symbol,Close,Previous_Day_Close,Next_Day_Close
0,2022-10-27,AAPL,144.800003,NaN,155.740005
1,2022-10-28,AAPL,155.740005,144.800003,153.339996
2,2022-10-31,AAPL,153.339996,155.740005,150.649994
3,2022-11-01,AAPL,150.649994,153.339996,145.029999
4,2022-11-02,AAPL,145.029999,150.649994,138.880005
...,...,...,...,...,...
1501,2023-10-20,NVDA,413.869995,421.010010,429.750000
1502,2023-10-23,NVDA,429.750000,413.869995,436.630005
1503,2023-10-24,NVDA,436.630005,429.750000,417.790009
1504,2023-10-25,NVDA,417.790009,436.630005,403.260010


### Rank stock prices per day

 To rank stock prices per day, you can use both the `RANK()` and `DENSE_RANK()` window functions. These functions assign a unique rank to each row within a result set based on the order you specify.

### SQL Query:
```sql
SELECT
  TradeDate,
  Symbol,
  Close,
  RANK() OVER (PARTITION BY TradeDate ORDER BY Close DESC) AS Rank_By_Close_Price,
  DENSE_RANK() OVER (PARTITION BY TradeDate ORDER BY Close DESC) AS Dense_Rank_By_Close_Price
FROM
  your_table_name
ORDER BY
  TradeDate, Rank_By_Close_Price;
```
Replace `your_table_name` with the actual name of your table.

### Step-by-Step Explanation:

1. **Window Functions**:
   - `RANK()`: This function assigns a unique rank to each row within the partition. If two (or more) rows have the same value in the `ORDER BY` clause, they will get the same rank, and the next rank will be skipped. For example, if two rows get a rank of 2, the next row will get a rank of 4.
   - `DENSE_RANK()`: This function also assigns a unique rank to each row within the partition. However, unlike `RANK()`, it does not skip any rank values. If two rows get a rank of 2, the next row will still get a rank of 3.

2. **Partitioning**: We partition the data by `TradeDate` to rank the closing prices separately for each day. This ensures that we're comparing stock prices from the same day.

3. **Ordering**: The rows are ordered by `Close` in descending order. This means the stock with the highest price on a given day gets rank 1.

In summary, for each stock symbol and date in the dataset, the query provides the closing price (`Close`), its rank for that day based on the closing price (`Rank_By_Close_Price`), and its dense rank for that day also based on the closing price (`Dense_Rank_By_Close_Price`). The difference between the two rank columns will be evident on days when two or more stocks have the same closing price; the `Rank_By_Close_Price` column will skip values, whereas the `Dense_Rank_By_Close_Price` column will not.

In [33]:
df = pd.io.gbq.read_gbq(f'''
SELECT
  TradeDate,
  Symbol,
  Close,
  RANK() OVER (PARTITION BY TradeDate ORDER BY Close DESC) AS Rank_By_Close_Price,
  DENSE_RANK() OVER (PARTITION BY TradeDate ORDER BY Close DESC) AS Dense_Rank_By_Close_Price
FROM
  {stocks_table}
ORDER BY
  TradeDate, Rank_By_Close_Price;
''', project_id=project_id)

df

,TradeDate,Symbol,Close,Rank_By_Close_Price,Dense_Rank_By_Close_Price
0,2022-10-27,NFLX,296.940002,1,1
1,2022-10-27,MSFT,226.750000,2,2
2,2022-10-27,AAPL,144.800003,3,3
3,2022-10-27,NVDA,131.759995,4,4
4,2022-10-27,META,97.940002,5,5
...,...,...,...,...,...
1501,2023-10-26,NVDA,403.260010,2,2
1502,2023-10-26,MSFT,327.890015,3,3
1503,2023-10-26,META,288.350006,4,4
1504,2023-10-26,AAPL,166.889999,5,5
